### Importing Required Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

### Importing Data

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train.reshape(x_train.shape[0],-1);  x_test = x_test.reshape(x_test.shape[0],-1);
y_train = y_train.reshape(-1,); y_test = y_test.reshape(-1,);

### Creating Model

In [66]:
model = keras.models.Sequential([
    keras.layers.Dense(50, activation = "relu"),
    keras.layers.Dense(50, activation = "relu"),
    keras.layers.Dense(50, activation = "relu"),
    keras.layers.Dense(10,  activation = "softmax")
])

class F_Score(tf.keras.metrics.Metric):

    def __init__(self, beta, name='F_score', **kwargs):
        
        super(F_Score, self).__init__(name=name, **kwargs)
        self.Beta = beta
        self.TP = self.add_weight(name='tp', initializer='zeros')
        self.FP = self.add_weight(name='fp', initializer='zeros')
        self.FN = self.add_weight(name='fn', initializer='zeros')
        self.F_score = self.add_weight(name='fs', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight = None):
        
        confusion_matrix = tf.math.confusion_matrix(y_true, tf.argmax(y_pred,axis=1))

        FP = tf.math.reduce_sum(confusion_matrix,axis=0) - tf.experimental.numpy.diag(confusion_matrix)  
        FP = tf.math.reduce_sum(FP)

        FN = tf.math.reduce_sum(confusion_matrix,axis=1) - tf.experimental.numpy.diag(confusion_matrix)  
        FN = tf.math.reduce_sum(FN)

        TP = tf.experimental.numpy.diag(confusion_matrix)
        TP = tf.math.reduce_sum(TP)
        
        self.TP.assign_add(tf.cast(TP,tf.float32))
        self.FN.assign_add(tf.cast(FN,tf.float32))
        self.FP.assign_add(tf.cast(FP,tf.float32))
        self.F_score.assign((1 + self.Beta**2)*self.TP/((1 + self.Beta**2)*self.TP + self.FP + (self.Beta**2)*self.FN))
        
    def result(self):
        return self.F_score

In [67]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(),
              optimizer = "Adam",
              metrics = [F_Score(beta = 1)])

In [68]:
model.fit(x_train, y_train, epochs = 3, validation_data = (x_test, y_test), verbose = 2, batch_size = 512)

Epoch 1/3
98/98 - 3s - loss: 17.6493 - F_score: 0.1589 - val_loss: 4.3395 - val_F_score: 0.1951 - 3s/epoch - 26ms/step
Epoch 2/3
98/98 - 2s - loss: 3.7982 - F_score: 0.2026 - val_loss: 4.0828 - val_F_score: 0.1973 - 2s/epoch - 17ms/step
Epoch 3/3
98/98 - 2s - loss: 3.1650 - F_score: 0.2167 - val_loss: 2.7427 - val_F_score: 0.2265 - 2s/epoch - 16ms/step
